# Calcualte performance rankings from cleaned results

In [1]:
# read cleaned results
import pandas as pd
from pathlib import Path
import pdb, os, sys
root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

agg_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_aggregated_results_with_default.csv"))
agg_df = pd.read_csv(Path("./cleaned_results/tuned_aggregated_results.csv"))

fold_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_fold_results_with_default.csv"))
fold_df = pd.read_csv(Path("./cleaned_results/tuned_fold_results.csv"))

# make sure output folder exists
output_folder = Path("./performance_rankings")
output_folder.mkdir(exist_ok=True)  

In [2]:
import numpy as np

def get_rank_table(df, metric, table_idx=4):
    """
    NOTE: the df needs to have the following columns defined:
    - {metric}_rank_mean
    - normalized_{metric}__test_mean
    """
    if table_idx == 1:
        from analysis.table1 import DATASETS
    elif table_idx == 2:
        from analysis.table2 import DATASETS
    elif table_idx == 4:
        from analysis.table4 import DATASETS
        
    df = df[df["dataset_name"].isin(DATASETS)]

    overall_ranks = df.groupby("alg_name").agg(
        {
            f"{metric}_rank_mean": ["min", "max", "mean", "median", "count"],
            f"normalized_{metric}__test_mean": ["mean", "median"],
            f"normalized_{metric}__test_std": ["mean", "median"],
            f"train_per_1000_inst_mean_{metric}": ["mean", "median"],        
        }
    ).reset_index().sort_values([(f"{metric}_rank_mean", "mean")])

    # format min/max rank columns to be ints

    overall_ranks.loc[:, (f"{metric}_rank_mean", "count")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "count")].astype(int)

    # overall_ranks.loc[:, "alg_name"] = overall_ranks.loc[:, "alg_name"].apply(lambda x: "\rot{" + x + "}")
    overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].apply(lambda x: "{:d}".format(int(x)))
    overall_ranks.loc[:, (f"{metric}_rank_mean", "max")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "max")].apply(lambda x: "{:d}".format(int(x)))

    # mean/median mean-rank
    overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"{metric}_rank_mean", "median")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "median")].apply(lambda x: "{:d}".format(int(x)) if int(x) == x else "{:.1f}".format(x))
    
    # normalized metric - mean and std over folds
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "median")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "median")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_std", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_std", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_std", "median")] = overall_ranks.loc[:,(f"normalized_{metric}__test_std", "median")].apply(lambda x: "{:.2f}".format(x))


    # normalized runtime
    overall_ranks.loc[:, (f"train_per_1000_inst_mean_{metric}", "mean")] = overall_ranks.loc[:,(f"train_per_1000_inst_mean_{metric}", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"train_per_1000_inst_mean_{metric}", "median")] = overall_ranks.loc[:,(f"train_per_1000_inst_mean_{metric}", "median")].apply(lambda x: "{:.2f}".format(x))
   


    final_table = overall_ranks.set_index("alg_name")

    return final_table


### Save rank tables to file

In [7]:
# save rank tables to csv and latex
from analysis_utils import ALG_DISPLAY_NAMES
table_idx = 4

metric_list = [
    "Accuracy",
    "F1",
    "Log Loss",
    "AUC",
]

final_tables = {}

# best, worst, and average performance for each alg, over all datasets
for metric in metric_list:

    agg_df_with_default

    # first with default hparams as its own alg
    final_tables[metric] = get_rank_table(agg_df_with_default, metric, table_idx = table_idx)

    # save to csv, latex
    final_tables[metric].to_csv(output_folder / f"{metric}_rank_with_default.csv", index=True)
    final_tables[metric].to_latex(output_folder / f"{metric}_rank_with_default.tex", index=True, escape=False)

    # now without default hparams as its own alg
    final_tables[metric] = get_rank_table(agg_df, metric, table_idx=table_idx)

    # save to csv, latex
    final_tables[metric].to_csv(output_folder / f"{metric}_rank.csv", index=True)
    final_tables[metric].to_latex(output_folder / f"{metric}_rank.tex", index=True, escape=False)

/tmp/ipykernel_3039144/3037057984.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' '1' '1' '1' '1' '1' '1' '3' '1' '2' '1' '1' '1' '1' '1' '3'
 '1' '1' '1' '7' '1' '1' '1' '3' '3' '1' '9' '6' '5' '2' '2' '5' '12' '6'
 '7' '4' '8' '6' '7' '5' '4' '8' '9' '8' '15' '16' '31']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].apply(lambda x: "{:d}".format(int(x)))
/tmp/ipykernel_3039144/3037057984.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['33' '44' '32' '41' '36' '38' '39' '33' '43' '39' '41' '33' '35' '40'
 '44' '41' '41' '42' '48' '48' '42' '36' '46' '46' '41' '41' '49' '40'
 '46' '41' '43' '41' '41' '46' '42' '43' '45' '46' '47' '49' '47' '48'
 '48' '49' '46' '49' '47' 

In [18]:
final_tables['AUC']

AUC_rank_mean                          \
                         min max   mean median count   
alg_name                                               
XGBoost                    1  19   4.58      2    36   
CatBoost                   1  18   5.31      4    35   
ResNet                     1  16   7.29      6    35   
LightGBM                   1  19   7.50    5.5    32   
SAINT                      1  23   8.19      7    27   
RandomForest               1  21   8.23      7    35   
Ours                       1  20   9.47     10    36   
DANet                      2  21  10.56     10    27   
FTTransformer              1  20  10.59     11    29   
TabPFN                     1  25  10.61     10    36   
MLP-rtdl                   1  22  10.64   10.5    36   
NODE                       2  24  10.77     12    30   
DeepFM                     2  20  10.86     11    21   
TabFast                    2  24  12.31   12.5    36   
TabNet                     1  23  13.36     13    33   
TabTransformer             1  25  13.46   14.5    26   
SVM                        1  22  13.52     16    29   
STG                        1  24  14.16     14    31   
TabFlex                    6  21  14.17     14    36   
MLP                        4  23  14.47     14    36   
LinearModel                5  25  15.59     16    34   
DecisionTree               5  25  16.83     17    35   
KNN                        5  25  17.61     18    33   
VIME                       7  24  17.84   19.5    32   
NAM                       13  25  20.94     22    16   

               normalized_AUC__test_mean        normalized_AUC__test_std  \
                                    mean median                     mean   
alg_name                                                                   
XGBoost                             0.94   0.98                     0.08   
CatBoost                            0.93   0.96                     0.09   
ResNet                              0.87   0.91                     0.09   
LightGBM                            0.92   0.95                     0.11   
SAINT                               0.80   0.91                     0.11   
RandomForest                        0.86   0.88                     0.09   
Ours                                0.76   0.87                     0.08   
DANet                               0.86   0.90                     0.11   
FTTransformer                       0.79   0.86                     0.11   
TabPFN                              0.64   0.82                     0.11   
MLP-rtdl                            0.78   0.84                     0.09   
NODE                                0.81   0.88                     0.11   
DeepFM                              0.83   0.88                     0.12   
TabFast                             0.65   0.78                     0.10   
TabNet                              0.77   0.86                     0.13   
TabTransformer                      0.68   0.80                     0.11   
SVM                                 0.76   0.78                     0.12   
STG                                 0.69   0.84                     0.13   
TabFlex                             0.62   0.74                     0.09   
MLP                                 0.67   0.69                     0.11   
LinearModel                         0.57   0.78                     0.10   
DecisionTree                        0.61   0.66                     0.12   
KNN                                 0.53   0.63                     0.13   
VIME                                0.58   0.49                     0.13   
NAM                                 0.45   0.53                     0.18   

                      train_per_1000_inst_mean_AUC          
               median                         mean  median  
alg_name                                                    
XGBoost          0.05                         1.82    0.29  
CatBoost         0.06                        51.02    1.47  
ResNet      

In [16]:
final_tables['AUC'][final_tables['AUC']['AUC_rank_mean']['count'] == 36]

AUC_rank_mean                         normalized_AUC__test_mean  \
                   min max   mean median count                      mean   
alg_name                                                                   
XGBoost              1  19   4.58      2    36                      0.94   
Ours                 1  20   9.47     10    36                      0.76   
TabPFN               1  25  10.61     10    36                      0.64   
MLP-rtdl             1  22  10.64   10.5    36                      0.78   
TabFast              2  24  12.31   12.5    36                      0.65   
TabFlex              6  21  14.17     14    36                      0.62   
MLP                  4  23  14.47     14    36                      0.67   

                normalized_AUC__test_std        train_per_1000_inst_mean_AUC  \
         median                     mean median                         mean   
alg_name                                                                       
XGBoost    0.98                     0.08   0.05                         1.82   
Ours       0.87                     0.08   0.05                         0.00   
TabPFN     0.82                     0.11   0.07                         0.00   
MLP-rtdl   0.84                     0.09   0.06                         6.84   
TabFast    0.78                     0.10   0.06                         0.00   
TabFlex    0.74                     0.09   0.09                         0.00   
MLP        0.69                     0.11   0.06                         8.84   

                 
         median  
alg_name         
XGBoost    0.29  
Ours       0.00  
TabPFN     0.00  
MLP-rtdl   4.29  
TabFast    0.00  
TabFlex    0.00  
MLP        5.42